# TODO
- we need to get more data into test the ideas - it should be easy to ingest data and iterating on the Task model might be the way to go
- API
- the project build and releast
- try benchmark datasets
- start the 20 posts and work through testing

## Status
 we have a pretty good model for types, functions, runners and postgres. We can tighten up the typing and generation of types 

## wishlist
 - add the duck and lance implmentations / redis for Entity store - and combine then into one embedded store
 - much richer postgres implementation
 - graph queries
 - dynamic type generation
 

# Big questions
What is the blocker and the friction; making it easy to add data that we can then re-organize feels important 

1. Combining text search e.g. find *all* TODOs
2. Making it easy to ingest data and intent
3. some sort of basic user interface to manage content
4. How to best use graphs - motif bombs 
5. Building structured data on background processes which also stresses the query planner (tasks and plans are an example if this as primaries)
6. adding the other agents and eval frameworks - measuring "prompt ambiguity"
7. parsers will be important - link managers, newsletter snippets

From this emerges blocks

- [A] Parsers and ingestors
- [B] Data modellers and depicters 
- [C] Query planners
- [D] General computer interfaces 

**Maybe i work on them in this order focusing on parsing out graph structures and entity ingestion**

In [ ]:
import funkyprompt
from IPython.display import Markdown
from funkyprompt.core import   load_entities
# from funkyprompt.services import language_model_client_from_context
# from funkyprompt.core.agents import MessageStack
from funkyprompt.core import AbstractEntity,AbstractModel, ConversationModel, AbstractContentModel
#Markdown(load_entities()[0].get_model_as_prompt())
# from funkyprompt.core.agents import CallingContext
from funkyprompt.core.agents import Runner
from funkyprompt.core.agents import QueryClassifier
# model = language_model_client_from_context()
# model('what is the capital of ireland')
#funkyprompt.ask("what is the capital of ireland")
#r = Runner()
#r('what is the capital of ireland')
#funkyprompt.run('what is the capital of ireland')
#funkyprompt.run('please ask for help and let me know what the status of planning in funkyprompt is')
#load_entities()
from funkyprompt.entities import Task

from funkyprompt.services import entity_store
#entity_store(Task).ask('id like to see task named Design101 ') #<-confidence in entity name
#entity_store(Task).ask('id like to see tasks that are in status TODO ') #<-confidence in entity status
#entity_store(Task).ask('id like to see example public.tasks ') #<-confidence query
#entity_store(Task).ask('id like to see tasks for funkyprompt') #<confidence vector search within neg inner prodict -0.79
from funkyprompt.services.data.lance import LanceDBService
from funkyprompt.core.utils import index_codebase, help, ingest_files
from funkyprompt.core.utils.parsing import scrape_text

#scrape_text('https://github.com/myscale/MyScaleDB')


# TODO 

- 1 . test the round trip of saving and loading the runner's type - check help model instances etc.
- 2 . test loading a different entity type and still having access to the methods on that entity - its a matter of function manager registration when the types are loaded via the store
- 3 . create upsert tests for different types of entities
- 4 . once we have done this, we can add the graph bomb - a graph bomb uses an abstract models to_graph to store structures
- 5 . this can all be put together with conversations and we can build concepts (stigmergic)

after we do this, we will the be generating lots of data that we can start testing query planning with and then function planning which requiers traversing over entity metadata

In [ ]:
from funkyprompt.entities import Project

r = Runner(Project)
"""this is a great and simple round trip test that i should keep - it requires the agent to load the entity and update it
but the magic is in how we register the entity and functions which has nothing to do with the agent per se.
"""
#r("can you add the label 'tested2' to the project entity called `test`")
#r("can you update the description for the project entity called `test` by adding the phrase, 'goodbye and thanks to all the fishes' at the end")

#add a note to the end of the description and save
r.lookup_entity('test')

In [ ]:
from inspect import getmembers, isfunction, isclass, ismethod, ismodule 
import types
import typing
from pydantic import BaseModel, Field
from enum import Enum
import importlib, pkgutil

def get_defining_class(member,cls):
    name = member.__name__
    defining_class = getattr(member, '__objclass__', None)
    if defining_class:
        return defining_class

    for base_class in cls.mro():
        if name in base_class.__dict__:
            return base_class
    return None
    
def is_strict_subclass(subclass, superclass):
    return issubclass(subclass, superclass) and subclass is not superclass

for name, member in getmembers(Project):
    if isfunction(member) or ismethod(member):
        defining_class = get_defining_class(member,Project)
        
        # Check if the method belongs to the class and not inherited
        
        print(name,member, defining_class, defining_class, is_strict_subclass(defining_class,AbstractModel))

In [ ]:
member.__name__

# questions
- entity loader should not load the embedding if its been selected via the entity 
- 

In [ ]:
 entity_store(Project).get_nodes_by_name('test')

In [ ]:
from funkyprompt.services import entity_store

entities =  entity_store(Project).get_nodes_by_name('test')
entities

In [ ]:
r.lookup_entity(key='test')

In [ ]:
#ingest_files("/Users/sirsh/Downloads/Roam-Export-1723329526148/", name='roam_notes')
M=AbstractContentModel.create_model(name='roam_notes', namespace='public')
#M._ask('are there any substack notes ?')
#index_codebase(provider=LanceDBService)
#help('How does the lance db service work - what way does it use the query classifier', provider=None, raw_search=False)
d=entity_store(AbstractContentModel).execute("select content,name from public.roam_notes  order by created_at desc limit 20")[17]
d

# What is the big idea
- we have a database of types including entities and edges
- now we need a smart way to map structured types to these underlying entities
- for example, if we have a subclass of category we may want to route it underlying types
- its unclear if we cheating too much here - might depend on the context
- start by seing how the agent maps the structures and then we can map it to the database by splitting out entities

In [ ]:
from funkyprompt.entities.nodes import TaskIdeaSummary
      
    
Markdown(TaskIdeaSummary.get_model_as_prompt())
        

In [ ]:
import json
R = Runner(TaskIdeaSummary)
Markdown(R.explain(d))

In [ ]:

Markdown(funkyprompt.run(f"""
1. What are some of the main ideas in the text below. please be detailed. 
2. also list the domain names (extract just the domain name without the full path first) and what you know about them in a list - include the full original link also - \n```{d}```
3. list all the real world entities mentioned in the text and what you know about them - you should inicate if its a specific entity e.g. person or thing or place OR if its a more general concept - you should provide also the category
4. Finally, considering my goals bellow, please provide tasks that link resources to goals. describe how ther resource could be used. Give an estimated utility score from 0 to 100 for the activity. Also describe the effort in terms of estimated days to do the work and how actionable it seems.
When creating tasks you can consider the type of activity a knowledge worker might have i.e. read or study, build or write, contract or network etc. It is ok to have multiple tasks for the same goal.

**my prioritized goals**
1. create a business for personal knowledge management AI with rich support for databases of different types such as key-value, sql, vetor and graph
2  Write as much as possible and finding good tools to manage my writing
3. undertand the challanges people face to day in terms of managing knowledge and personal growth
4. learning new AI methods and data modeling methods
5. possible integrations from popular services and my app
  
"""))

In [ ]:
from urllib.parse import urlparse
p=urlparse('https://github.com/myscale/MyScaleDB')
p

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://github.com/myscale/MyScaleDB'
url = 'https://raw.githubusercontent.com/myscale/MyScaleDB/main/README.md'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup#Markdown(soup.get_text())

In [ ]:
response

In [ ]:
Markdown(entity_store(AbstractContentModel).execute("select content,name from public.roam_notes  order by created_at desc limit 10")[3]['content'])

In [ ]:
from funkyprompt.core.agents import CallingContext, LanguageModel, MessageStack
from funkyprompt.core.agents.MessageStack import SystemMessage
import json
data = {'asdf':'asta'}
messages= [
            SystemMessage(content=f'The data below were retrieved from a resource and can be used to answer the users question\n````{json.dumps(data,default=str)}``')
        ]
question = 'test'
MessageStack(question=question,messages=messages, model=AbstractContentModel)

In [ ]:
# a= AbstractContentModel.create_model(name='codebase', namespace='core')

# a._register()

In [ ]:
from funkyprompt.core.utils import index_codebase, help
help('How can this library help me with buildint AI agents?', raw_search=False)
#index_codebase()

In [ ]:
#QueryClassifier._classify_question_as_query_for_model(model=Task,question='id like to see task Design101 and all tasks that were completed in the last month',preview=False)
#Markdown(QueryClassifier._classify_question_as_query_for_model(model=Task,question='id like to see task Design101 and all tasks that were completed in the last month',preview=False, return_model=False))

In [ ]:
#Markdown(QueryClassifier._classify_question_as_query_for_model(model=Task,question='id like to see all my tasks with status as TODO',preview=False, return_model=False))
Markdown(QueryClassifier._classify_question_as_query_for_model(model=Task,question='id like to see tasks for funkyprompt',preview=False, return_model=False))

In [ ]:
#Markdown(QueryClassifier._classify_question_as_query_for_model(model=Task,question='id like to see all my tasks related to programming and where complex functionl langauge are discussed',preview=False, return_model=False))

In [ ]:


#Task._register()

t = Runner(Task)
#t("Can you create a task called Design101. This is for designing the funkybrain project which will be a reference implementation for funkyprompt. Add labels such as AI, RAG and Postgres to it and Id like to complete it by next week",limit=2)
#t('can you set the task Design101 due date to next week',limit=2)
t('can you list my tasks',limit=2)

In [ ]:
t("What functions do you have to change task status?")

In [ ]:
from funkyprompt.services import entity_store
entity_store(Task).get_nodes_by_name('Design101')

In [ ]:
from funkyprompt.services import entity_store
from funkyprompt.core import AbstractModel
#entity_store(AbstractModel).execute('select * from information_schema.tables')

In [ ]:
# entity_store(AbstractModel).__drop_table__()
# entity_store(AbstractModel)._create_model()

In [ ]:
#ConversationModel._register()

In [ ]:
funkyprompt.run('please ask for help and let me know what the status of planning in funkyprompt is')

In [ ]:
# from funkyprompt.entities import Project
# from funkyprompt.core import ConversationModel
# from funkyprompt.services import entity_store
# store =entity_store(Project)
# store.vector_search('what projects tell us about sirsh interests')
# store =entity_store(ConversationModel)
# store.vector_search('what did i asked about installing postgres on the mac')
# #store.vector_search('{"question": "please ask for help and let me know what the status of planning in funkyprompt is", "The status of planning in Funkyprompt is currently pending. It is suggested to use world knowledge for now. If you have any specific questions or need assistance with something, feel free to ask!": "The status of planning in Funkyprompt is currently pending. It is suggested to use world knowledge for now. If you have any specific questions or need assistance with something, feel free to ask!"}')

In [ ]:
#store.update_records(p)

In [ ]:
# Entity state - the task


In [ ]:
# advanced - thoughts/currents/states/learning
# using graph links and duck aggregate types

# objectives, current thinking, priorisation

#funkybrain.ask('what am i doing this week')
#funkybrain.ask('what should i do this week')
#funkybrain.ask('what should i do today')

#funkybrain.summarize(SummaryModel) #-> a store reorganized

# changing state is either a way to upsert or change attributes on records or a separate summary record that mutates 
##. - like a favourites links that can be added or dissolved
# therefore a node like 'today' or 'this week' which would actually be dynamically resoled can have links 
#...link bulder agent
#...maybe a summary node should always be done like this i.e. not on the same grain as flat records 
